# Federated Learning

### How does the code work?

* Explained in this tutorial through DataCamp:
[Federated Learning](https://www.datacamp.com/blog/federated-learning)

#### 1. Initialization Phase:

A global model was created with random weights (weight1, bias1, etc.).
This global model was distributed to all clients.

#### 2. Local Training Phase:

Each client trained its local version of the global model using its private data.
Training involved updating the model weights based on simulated gradient descent.
After training, each client returned its updated model parameters (the "local updates").

#### 3. Aggregation Phase:
The server aggregated the updates from all clients using federated averaging.

In [ ]:
import numpy as np

# Class for Initialization Phase
class InitializationPhase:
    def __init__(self, model, hyperparameters):
        """
        Initialize the global model and distribute it to clients.

        Parameters:
        model: Initial global model (a dictionary of parameters or weights)
        hyperparameters: A dictionary of hyperparameters (e.g., learning rate, epochs)
        """
        self.global_model = model
        self.hyperparameters = hyperparameters

    def distribute_model(self, clients):
        """
        Distribute the global model and hyperparameters to all clients.

        Parameters:
        clients: List of client instances
        """
        for client in clients:
            client.set_model(self.global_model, self.hyperparameters)

In [ ]:
# Class for Local Training Phase
class LocalTraining:
    def __init__(self, local_data):
        """
        Initialize local data for the client.

        Parameters:
        local_data: Local dataset for training (features and labels)
        """
        self.local_data = local_data
        self.model = None
        self.hyperparameters = None

    def set_model(self, model, hyperparameters):
        """
        Receive global model and hyperparameters from the server.

        Parameters:
        model: The global model received from the server
        hyperparameters: Training configuration
        """
        self.model = model
        self.hyperparameters = hyperparameters

    def train(self):
        """
        Train the local model on the local dataset.
        """
        epochs = self.hyperparameters['epochs']
        learning_rate = self.hyperparameters['learning_rate']

        # Simulated training (gradient descent updates)
        for epoch in range(epochs):
            gradients = self.compute_gradients(self.local_data, self.model)
            self.model = {key: value - learning_rate * gradients[key] for key, value in self.model.items()}

        return self.model

    def compute_gradients(self, data, model):
        """
        Compute gradients for the model parameters using the local data.
        (This is a placeholder; replace with actual gradient computation logic.)

        Parameters:
        data: Local dataset
        model: Current model parameters

        Returns:
        Gradients for the model parameters
        """
        gradients = {key: np.random.randn(*value.shape) for key, value in model.items()} # Dummy gradients
        return gradients

In [ ]:
# Class for Aggregation Phase
class AggregationPhase:
    @staticmethod
    def aggregate_updates(client_updates):
        """
        Aggregate updates from all clients using federated averaging.

        Parameters:
        client_updates: List of updated model parameters from clients

        Returns:
        Aggregated global model
        """
        aggregated_model = {}

        # Average updates from all clients
        for key in client_updates[0].keys():
            aggregated_model[key] = np.mean([update[key] for update in client_updates], axis=0)

        return aggregated_model

In [ ]:

if __name__ == "__main__":
    # Simulated global model (a dictionary of weights)
    global_model = {
        'weight1': np.random.randn(10, 10),
        'bias1': np.random.randn(10),
        'weight2': np.random.randn(10, 1),
        'bias2': np.random.randn(1)
    }

    # Hyperparameters for training
    hyperparameters = {
        'learning_rate': 0.01,
        'epochs': 5
    }

    # Simulated local datasets for clients (in random)
    client_datasets = [
        (np.random.randn(100, 10), np.random.randn(100)),
        (np.random.randn(100, 10), np.random.randn(100)),
        (np.random.randn(100, 10), np.random.randn(100))
    ]

    # Create InitializationPhase and distribute the global model to clients
    init_phase = InitializationPhase(global_model, hyperparameters)
    clients = [LocalTraining(data) for data in client_datasets]
    init_phase.distribute_model(clients)

    # Perform local training on all clients
    client_updates = []
    for client in clients:
        updated_model = client.train()
        client_updates.append(updated_model)

    # Aggregate updates and create a new global model
    aggregated_model = AggregationPhase.aggregate_updates(client_updates)

    print("New global model:", aggregated_model)

New global model: {'weight1': array([[-2.39940396e-01, -2.88164898e-01, -1.77012167e+00,
        -9.67319409e-01, -1.98182237e-01,  9.59768504e-02,
        -1.14915973e-01, -7.56992990e-01, -2.90372413e-01,
        -1.28187603e+00],
       [-1.24942784e+00, -3.39066452e-01,  2.79349103e-01,
        -6.06846405e-01, -9.99616688e-02, -2.06621358e-01,
        -1.10919951e+00, -7.35665036e-01, -2.99234804e-01,
         1.22595632e+00],
       [-9.86264910e-01, -4.36179553e-02, -8.09145038e-01,
        -7.93534466e-01,  2.10178219e-01,  8.15231567e-01,
         4.98967190e-01, -3.89971946e-01, -1.07285947e-01,
         1.46259648e-01],
       [ 4.33666371e-01, -1.34307907e+00, -1.54323223e+00,
        -1.93957334e+00,  1.42626879e+00,  6.87892174e-02,
         4.93167756e-01,  4.06908600e-01,  3.25623782e-01,
         7.51650034e-02],
       [ 1.22962616e+00, -3.71498790e-01, -4.79777557e-02,
         8.29505241e-01, -8.95509304e-01,  7.99148472e-01,
        -6.09659689e-01,  1.27328127e+00